In [120]:
# why need to increase batch size for higher k?

import numpy as np
from sklearn.datasets import make_blobs
from sklearn.metrics import pairwise_distances_argmin
# https://scikit-learn.org/stable/modules/generated/sklearn.metrics.pairwise_distances_argmin.html
from time import time
import random
np.random.seed(42)
random.seed(42)

X, y_true = make_blobs(n_samples=1500, centers=4, cluster_std=0.60, random_state=0)

# mini-batch for faster computation
class Mini_KMeans:
    def __init__(self, k, replacement=True, batch_size=500, max_iter=100):
        self.k = k
        self.replacement=replacement
        self.batch_size = batch_size
        self.max_iter = max_iter
        
    def fit(self, X):
        m, n = X.shape
        i = np.random.permutation(m)[:self.k]
        print('i:',i)
        self.centers = X[i]
        print('\nfor', self.k, 'initial centers: \n',self.centers,'\n')
        
        #having max iter makes sure it will stop eventually
        for ix in np.arange(self.max_iter):
            random_num = random.randint(0, m)
            X_batch = X[random_num:random_num+self.batch_size]
            labels = pairwise_distances_argmin(X_batch, self.centers)

#             print('X_batch[:5]\n',X_batch[:5])
#             print('\nself.centers\n',self.centers)
#             print('\nlabels_:\n',labels_[:5])
#             print('\nlabels:\n',labels[:5],'\n')

            new_centers = []
            for i in range(self.k):
                new_centers.append(X_batch[labels == i].mean(axis=0))
            new_centers = np.array(new_centers)
            
            if(np.allclose(self.centers, new_centers, rtol=0.2)):
                break
            else:
                self.centers = new_centers
        
        print(ix, 'iterations required')

        #compute total within-variation score
        total_with_variation_score = 0
        labels = pairwise_distances_argmin(X, self.centers) #<---Note I use X here.  Why?
        for i in range(self.k):
            cluster_mean = X[labels==i].mean(axis=0)
            total_with_variation_score += ((X[labels==i] - cluster_mean)** 2).sum()
            
        print("Total with variation score: ", total_with_variation_score)
        
        
    def predict(self, X):
        return pairwise_distances_argmin(X, self.centers, metric='euclidean')
    
for k in range(1, 11):
    model = Mini_KMeans(k)
    model.fit(X)
    preds = model.predict(X)
    print(preds.shape)

i: [1116]

for 1 initial centers: 
 [[-1.45375109  7.23580201]] 

8 iterations required
Total with variation score:  14005.126690305286
(1500,)
i: [479 722]

for 2 initial centers: 
 [[ 0.37614087  3.37692467]
 [-1.89955363  3.95862557]] 

1 iterations required
Total with variation score:  7143.275336599341
(1500,)
i: [1068   59 1095]

for 3 initial centers: 
 [[ 1.95433899  1.12440972]
 [-1.72767698  2.390148  ]
 [ 0.13269233  3.94975274]] 

1 iterations required
Total with variation score:  3917.389048840629
(1500,)
i: [739 296 863 254]

for 4 initial centers: 
 [[ 2.1077864   1.46091179]
 [-0.6409548   8.66597205]
 [ 0.75745395  4.39780964]
 [-1.31461008  7.57600083]] 

1 iterations required
Total with variation score:  2422.162702505687
(1500,)
i: [1307  741  135  554  279]

for 5 initial centers: 
 [[-1.49922298  2.34285757]
 [-1.87736014  3.11894563]
 [ 2.31659522  0.53812909]
 [ 1.91168957  0.74958753]
 [-1.46655529  3.24137903]] 

7 iterations required
Total with variation scor